In [221]:
import pandas as pd
data = pd.read_csv('archive/Food Ingredients and Recipe Dataset with Image Name Mapping.csv')

In [1]:
import pandas as pd
data = pd.read_json('archive/allrecipes-recipes.json.xz',lines=True, compression='xz')


In [2]:
data.head()

,author,cook_time_minutes,description,error,footnotes,ingredients,instructions,photo_url,prep_time_minutes,rating_stars,review_count,time_scraped,title,total_time_minutes,url
0,Stephanie,25,I just started adding my favorite things to ba...,False,[],"[1/2 cup unsalted butter, chilled and cubed, 1...",[Preheat oven to 400 degrees F (205 degrees C)...,http://images.media-allrecipes.com/userphotos/...,55,4.32,46,1498204021,"Basil, Roasted Peppers and Monterey Jack Cornb...",100,http://allrecipes.com/Recipe/6664/
1,Stephanie,0,These are great as an appetizer or served alon...,False,[],"[1/2 cup Parmesan cheese, 3/4 teaspoon ground ...","[Combine parmesan cheese, pepper and garlic po...",http://images.media-allrecipes.com/userphotos/...,0,4.18,44,1498204301,Crispy Cheese Twists,0,http://allrecipes.com/Recipe/6663/
2,Kathy,0,This is the best bread recipe. Light and fluf...,False,[],"[2 cups hot water, 1/2 cup margarine, 1/3 cup ...",[Melt margarine in hot water. Add sugar and sa...,http://images.media-allrecipes.com/userphotos/...,0,3.65,168,1498206088,Mom's Yeast Rolls,0,http://allrecipes.com/Recipe/6665/
3,Mary E. Crain,0,A Southern delight with a 'more-ish' taste. ...,False,[],"[1 1/2 cups white sugar, 1/2 cup vegetable oil...",[Combine sugar and oil; beat well. Add eggs an...,http://images.media-allrecipes.com/userphotos/...,0,4.70,344,1498209193,Sweet Potato Bread I,0,http://allrecipes.com/Recipe/6666/
4,DeeDee,45,You can make these into braids or buns. Frost ...,False,"[Cook's Note:, I made 1 braid, 12 cinnamon bun...","[1/4 cup butter, 1 teaspoon white sugar, 1 cup...",[Stir butter and 1 teaspoon sugar into the hot...,http://images.media-allrecipes.com/userphotos/...,20,4.71,6,1498205251,Orange Buns,170,http://allrecipes.com/Recipe/6668/


In [160]:
'''
convert_to_list = ['instructions', 'ingredients']
for c in convert_to_list:
    data[c] = data[c].apply(lambda x: list(x))
'''
data.to_pickle('data/raw_df.pkl')

In [3]:
import re
def clean_dataset(dataset):
    def remove_values(x):
        l = [re.sub("\d+\/*\d*", "", str(i)) for i in x]
        l = [re.sub("\\+", "", str(i)) for i in l]
        prohibitedWords = ['cups', 'cup', 'teaspoons', 'teaspoon', 'tablespoons', 'tablespoon', 'pounds', 'pound', 'ounces', 'ounce', 'and']
        big_regex = re.compile('|'.join(map(re.escape, prohibitedWords)))
        l = [big_regex.sub("", t) for t in l]
        return l
    data = dataset.assign(ingredients_clean = lambda x: remove_values(x['ingredients']))
    return data

In [4]:
data = clean_dataset(data)
data.ingredients_clean

0         ['  unsalted butter, chilled  cubed', '  chopp...
1         ['  Parmesan cheese', '  ground black pepper',...
2         ['  hot water', '  margarine', '  white sugar'...
3         ['   white sugar', '  vegetable oil', ' eggs',...
4         ['  butter', '  white sugar', '  hot milk', ' ...
                                ...                        
225597    ['cooking spray', '  milk', '  white sugar', '...
225598    ['  lean ground turkey', ' onion, diced', '  c...
225599    ["  cassava flour (such as Otto's)", '  unswee...
225600    [' ( ) can pinto beans, rinsed  drained', '  u...
225601    ['Cream:', '  heavy whipping cream', ' small c...
Name: ingredients_clean, Length: 225602, dtype: object

In [5]:
import gensim
from sklearn.feature_extraction.text import CountVectorizer
def topic_modeling(data):
    ingredients_all = data.ingredients_clean
    vect = CountVectorizer(token_pattern='(?u)\\b\\w\\w\\w+\\b')
    X = vect.fit_transform(ingredients_all)
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    id_map = dict((v, k) for k, v in vect.vocabulary_.items())
    print(len(id_map))
    ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics=50,passes=1, chunksize=7500, random_state=0, id2word = id_map)
    return ldamodel

In [6]:
ldamodel = topic_modeling(data)

7003


In [7]:
ldamodel.print_topics(num_topics = 50,num_words = 15)

[(0,
  '0.167*"chopped" + 0.094*"fresh" + 0.078*"finely" + 0.035*"dill" + 0.031*"leaves" + 0.031*"optional" + 0.029*"lemon" + 0.028*"mint" + 0.023*"juice" + 0.023*"garnish" + 0.019*"cabbage" + 0.019*"sprigs" + 0.017*"hard" + 0.017*"for" + 0.015*"bunch"'),
 (1,
  '0.041*"garlic" + 0.040*"minced" + 0.040*"oil" + 0.039*"sauce" + 0.037*"taste" + 0.035*"pepper" + 0.031*"salt" + 0.028*"sliced" + 0.028*"ground" + 0.028*"cloves" + 0.027*"soy" + 0.025*"fresh" + 0.024*"chopped" + 0.024*"black" + 0.018*"olive"'),
 (2,
  '0.120*"spray" + 0.106*"cooking" + 0.093*"mix" + 0.073*"package" + 0.052*"divided" + 0.048*"instant" + 0.045*"cake" + 0.034*"milk" + 0.029*"oil" + 0.028*"water" + 0.027*"chocolate" + 0.027*"eggs" + 0.024*"vegetable" + 0.022*"pudding" + 0.021*"coffee"'),
 (3,
  '0.185*"asparagus" + 0.086*"pomegranate" + 0.080*"skin" + 0.078*"spears" + 0.072*"with" + 0.049*"calorie" + 0.044*"gum" + 0.043*"stuffed" + 0.030*"spiced" + 0.028*"breaded" + 0.025*"ravioli" + 0.024*"pimento" + 0.023*"drumst

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
ingredients_all = data.ingredients_clean
tfidf_vectorizer = TfidfVectorizer(max_df=0.99, max_features=None, token_pattern='(?u)\\b\\w\\w\\w+\\b')
tfidf = tfidf_vectorizer.fit_transform(ingredients_all)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

In [9]:
n_topics = 300
n_top_words = 20

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_[:10]): # just show first 10 topics
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    
import operator
def rank_terms(A, terms):
    sums = A.sum(axis=0)
    weights = {}
    for col, term in enumerate(terms):
        weights[term] = sums[0,col]
    return sorted(weights.items(), key=operator.itemgetter(1), reverse=True)

In [10]:
ranking = rank_terms(tfidf, tfidf_feature_names)
ranking

[('pizza', 50375.604795809806),
 ('package', 47612.49500755471),
 ('sliced', 47456.44836988155),
 ('cheese', 46589.84790967836),
 ('three', 25246.714867031584),
 ('johnsonville', 25244.387821079268),
 ('pre', 25241.08812191102),
 ('pepperoni', 25231.993093129033),
 ('baked', 25206.9595625014),
 ('crust', 25091.689702540538),
 ('style', 25087.81074447101),
 ('sausage', 25079.305681100668),
 ('jar', 25072.662324812452),
 ('mozzarella', 25072.559916989525),
 ('italian', 24995.87530751257),
 ('shredded', 24543.583205359908),
 ('inch', 24536.131734907947),
 ('chicken', 24464.653539076673),
 ('sauce', 24452.953250205945),
 ('chopped', 6484.61976747412),
 ('sugar', 5182.384098705408),
 ('ground', 5001.315376911083),
 ('salt', 4965.784902546231),
 ('pepper', 4727.658892353473),
 ('butter', 4063.0254304720147),
 ('white', 3760.476296119518),
 ('fresh', 3669.162269009908),
 ('oil', 3559.5450481370685),
 ('flour', 3391.6193334988034),
 ('taste', 3224.412454284658),
 ('can', 3119.478937839081),
 (

In [15]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=50, random_state=1, l1_ratio=.5, verbose=2, max_iter=100).fit(tfidf)


C:\Users\Shru\anaconda3\envs\cs7643-a4\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


violation: 1.0
violation: 0.3448238253183641
violation: 0.2724703386098828
violation: 0.21162862435752913
violation: 0.1650603629352824
violation: 0.13399396482999967
violation: 0.11112035384749189
violation: 0.0879545937553656
violation: 0.06600335324803147
violation: 0.05051052857114766
violation: 0.04096388868441268
violation: 0.03462329415947828
violation: 0.03091668376759594
violation: 0.028756803343785702
violation: 0.02756293311552833
violation: 0.026678289001082753
violation: 0.025221148326491204
violation: 0.02268391474770918
violation: 0.018984989472348626
violation: 0.015594159480783408
violation: 0.013188491771167725
violation: 0.011235786998148652
violation: 0.00971618718602226
violation: 0.008684084594717397
violation: 0.007907994199976018
violation: 0.00739002561150729
violation: 0.007079620943050324
violation: 0.006921653173535882
violation: 0.006887270740303712
violation: 0.007002213934568924
violation: 0.007182886542963327
violation: 0.0074797037419505625
violation: 0

In [17]:
print_top_words(nmf, tfidf_feature_names, 15)

Topic #0:
pizza package sliced cheese three johnsonville pre pepperoni baked style crust sausage mozzarella jar italian
Topic #1:
chopped onion celery walnuts coarsely pecans tomatoes parsley small broth cooked carrots stalks cilantro spinach
Topic #2:
all purpose flour salt shortening butter sugar sifted cold needed milk baking unbleached for cied
Topic #3:
can drained beans rinsed condensed tomatoes cans corn soup kidney with green pineapple chili undrained
Topic #4:
fresh parsley leaves mushrooms basil mint sprigs thyme strawberries rosemary chives cilantro blueberries ginger trimmed
Topic #5:
ground cinnamon nutmeg ginger cloves allspice raisins cumin pumpkin molasses apples puree pinch corier apple
Topic #6:
into cut pieces inch cubes strips chunks small bite wedges size thin thick half dice
Topic #7:
pepper black ground salt cayenne paprika freshly pinch onion garlic kosher pork hot potatoes dash
Topic #8:
lemon juice zest peel juiced fillets salmon grated mayonnaise pineapple di

#Ignore Below using bag of words

In [118]:
ingredients_all = data.ingredients
ing = []
print(len(ingredients_all))
for i in ingredients_all:
    l = [re.sub("\d+\/*\d*", "", str(v)).strip() for v in i]
    prohibitedWords = ['cups', 'cup', 'teaspoons', 'teaspoon', 'tablespoons', 'tablespoon', 'pounds', 'pound', 'ounces', 'ounce']
    big_regex = re.compile('|'.join(map(re.escape, prohibitedWords)))
    l = [big_regex.sub("", t).strip() for t in l]
    ing.append(l)
ing[:20]
dictionary = corpora.Dictionary(ing)
corpus = [dictionary.doc2bow(text) for text in ing]
ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics=25,passes=1, chunksize=7500, random_state=0, id2word = dictionary)
ldamodel.print_topics(num_topics = 25,num_words = 15)

225602


[(0,
  '0.080*"white sugar" + 0.072*"large eggs" + 0.054*"water" + 0.053*"white vinegar" + 0.047*"salt" + 0.039*"pinch salt" + 0.035*"Topping:" + 0.029*"all-purpose flour" + 0.027*"egg, beaten" + 0.024*"butter or margarine" + 0.024*"firmly packed brown sugar" + 0.020*"milk" + 0.020*"light corn syrup" + 0.019*"cornmeal" + 0.017*"vanilla extract"'),
 (1,
  '0.101*"( ) package cream cheese, softened" + 0.042*"% milk" + 0.039*"( ) can sweetened condensed milk" + 0.031*"( ) package cream cheese" + 0.031*"pinch cayenne pepper, or to taste" + 0.029*"( ) container sour cream" + 0.028*"cold milk" + 0.027*"tahini" + 0.022*"diced green bell pepper" + 0.021*"( ) container frozen whipped topping, thawed" + 0.020*"bananas" + 0.020*"oil for frying" + 0.018*"light soy sauce" + 0.017*"butter, melted" + 0.017*"rice wine"'),
 (2,
  '0.150*"salt and ground black pepper to taste" + 0.061*"garlic powder" + 0.051*"olive oil" + 0.044*"onion powder" + 0.036*"lemon, juiced" + 0.036*"ground black pepper" + 0.035